In [2]:
# !pip install keras-cv
# !pip install dill

In [3]:
# Importing dependencies
import os
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.optimizers import AdamW
import keras_cv


BATCH_SIZE = 32
GLOBAL_CLIPNORM = 10.0
bounding_box_format = "center_xywh"
AUTO = tf.data.AUTOTUNE
EPOCHS=1

D:\chientuhocai\Traffic_Sign\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

# 1. Preprocessing

In [2]:
# a function for converting txt file to list
def parse_txt_annot(img_path, txt_path):
    #read image and get shape
    img = cv2.imread(img_path)
    w = int(img.shape[0])
    h = int(img.shape[1])
    
    #read label file
    file_label = open(txt_path, "r")
    lines = file_label.read().split('\n')

    boxes = []
    classes = []

    for i in range(0, int(len(lines))):
        # check line is empty or not
        if lines[i] == '':
            continue
        objbud=lines[i].split(' ')
        try:
            class_ = int(objbud[0])
            x1 = float(objbud[1])
            y1 = float(objbud[2])
            w1 = float(objbud[3])
            h1 = float(objbud[4])
        except Exception as e:
            print(e, txt_path)
            continue
        # convert to x_center, y_center, width, height
        x_center = int((x1 * w))
        y_center = int((y1 * h))
        width = int((w1 * w))
        height = int((h1 * h))
        
        boxes.append([x_center , y_center , width , height])
        classes.append(class_)

    return img_path, classes, boxes


# a function for creating file paths list
def create_paths_list(path):
    full_path = []
    images = sorted(os.listdir(path))

    for i in images:
        full_path.append(os.path.join(path, i))

    return full_path


class_ids = class_labels = [
    "sign",
]

class_mapping = dict(zip(range(len(class_ids)), class_ids))
num_classes = len(class_ids)
num_classes

1

In [6]:
# a function for creating a dict format of files
def creating_files(img_files_paths, annot_files_paths):
    # get list of image and annotation files
    img_files = create_paths_list(img_files_paths)
    annot_files = create_paths_list(annot_files_paths)

    image_paths = []
    bbox = []
    classes = []

    for i in range(0,len(img_files)):
        try:
            # parse txt annotation file - get image path, classes and bounding boxes
            image_path_, classes_, bbox_ = parse_txt_annot(img_files[i], annot_files[i])
            image_paths.append(image_path_)
            bbox.append(bbox_)
            classes.append(classes_)
        except Exception as e:
            # remove the image and annotation file if there is an error
            print(e, img_files[i])
            continue

    image_paths = tf.ragged.constant(image_paths)
    bbox = tf.ragged.constant(bbox)
    classes = tf.ragged.constant(classes)

    return image_paths, classes, bbox

In [7]:
# orin_path='/kaggle/input/detection-traffic-sign-mix/dataset'
orin_path='../dataset/final_detection'


In [8]:
# applying functions
train_img_paths, train_classes, train_bboxes = creating_files(f'{orin_path}/train/images',
                                                              f'{orin_path}/train/labels')
print("Finished creating training files")
valid_img_paths, valid_classes, valid_bboxes = creating_files(f'{orin_path}/valid/images',
                                                              f'{orin_path}/valid/labels')
print('Finished creating validation files')
test_img_paths, test_classes, test_bboxes = creating_files(f'{orin_path}/test/images',
                                                           f'{orin_path}/test/labels')
print('Finished creating testing files')

list index out of range ../dataset/final_detection/train/images\stop_1323820809-avi_image9_png.rf.c72c56f427902835f9f94a8ea1cb6431.jpg
Finished creating training files
Finished creating validation files
Finished creating testing files


## Creating Dataset

In [9]:
# reading and resizing images
def img_preprocessing(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.cast(img, tf.float32)
    return img


# JitteredResize for data augmentation
resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format=bounding_box_format)

# loading dataset
def load_ds(img_paths, classes, bbox):
    img = img_preprocessing(img_paths)

    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox }

    return {"images": img, "bounding_boxes": bounding_boxes}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

In [10]:
# Creating dataset loaders and tf.datasets
train_loader = tf.data.Dataset.from_tensor_slices((train_img_paths, train_classes, train_bboxes))
train_dataset = (train_loader
                 .map(load_ds, num_parallel_calls = AUTO)
                 .shuffle(BATCH_SIZE*10)
                 .ragged_batch(BATCH_SIZE, drop_remainder = True)
                 .map(resizing, num_parallel_calls = AUTO)
                 .map(dict_to_tuple, num_parallel_calls = AUTO)
                 .prefetch(AUTO))

print("Finished creating training dataset")
valid_loader = tf.data.Dataset.from_tensor_slices((valid_img_paths, valid_classes, valid_bboxes))
valid_dataset = (valid_loader
                 .map(load_ds, num_parallel_calls = AUTO)
                 .ragged_batch(BATCH_SIZE, drop_remainder = True)
                 .map(resizing, num_parallel_calls = AUTO)
                 .map(dict_to_tuple, num_parallel_calls = AUTO)
                 .prefetch(AUTO))

print("Finished creating validation dataset")
test_loader = tf.data.Dataset.from_tensor_slices((test_img_paths, test_classes, test_bboxes))
test_dataset = (test_loader
                .map(load_ds, num_parallel_calls = AUTO)
                .ragged_batch(BATCH_SIZE, drop_remainder = True)
                .map(resizing, num_parallel_calls = AUTO)
                .map(dict_to_tuple, num_parallel_calls = AUTO)
                .prefetch(AUTO))
print("Finished creating testing dataset")

Finished creating training dataset
Finished creating validation dataset
Finished creating testing dataset


In [11]:
def preprocess_data(images, labels):
    # Trích xuất classes và boxes từ labels
    classes = labels['classes']
    boxes = labels['boxes']

    # Chuyển đổi RaggedTensor thành Tensor với padding
    classes = classes.to_tensor(default_value=-1.0)  # Padding classes
    boxes = boxes.to_tensor(default_value=0.0)      # Padding boxes

    # Đảm bảo boxes có shape (batch_size, num_boxes, 4)
    boxes = tf.reshape(boxes, (tf.shape(boxes)[0], -1, 4))

    # Đảm bảo classes có shape (batch_size, num_boxes)
    classes = tf.reshape(classes, (tf.shape(classes)[0], -1))

    # Trả về images và labels đã xử lý
    return images, {'classes': classes, 'boxes': boxes}

# Áp dụng hàm lên dataset
train_dataset = train_dataset.map(preprocess_data)
valid_dataset = valid_dataset.map(preprocess_data)
test_dataset= test_dataset.map(preprocess_data)

In [ ]:
# a function to visualize samples from a dataset
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs[0], inputs[1]

    keras_cv.visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale = 8,
        font_scale = 0.8,
        line_thickness=2,
        dpi = 100,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
        true_color = (192, 57, 43))

In [ ]:
# examples images and annotations from training daatset
visualize_dataset(valid_dataset, bounding_box_format=bounding_box_format, value_range=(0, 255), rows=2, cols=2)

# 2. Model

In [12]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))  # Kiểm tra GPU

# Thiết lập mixed precision để tăng tốc độ trên GPU (tùy chọn)
tf.keras.mixed_precision.set_global_policy('float32')  #

Num GPUs Available:  0


In [1]:
def create_model(
    num_classes,
    input_shape=(640, 640, 3),  # Định dạng chuẩn cho YOLO
    fpn_depth=1,
    learning_rate=0.001,
    weight_decay=0.004,
    use_pretrained=True,  # Thêm flag để kiểm soát việc sử dụng pre-trained
    trainable=False,  # Thêm flag để kiểm soát việc đóng băng backbone
    iou_threshold=0.7
):

    # Tải pre-trained backbone ----------------------------------------------------
    backbone = keras_cv.models.YOLOV8Backbone.from_preset(
        "yolo_v8_xs_backbone_coco",  # Pre-trained trên COCO
        input_shape=input_shape
    )
    print("Loaded pre-trained YOLOv8 backbone")

    # Define custom NMS decoder with IoU threshold
    prediction_decoder = keras_cv.layers.MultiClassNonMaxSuppression(
        iou_threshold=iou_threshold,              # Your IoU threshold (default: 0.7)
        confidence_threshold=0.5,       # Minimum confidence to keep predictions
        bounding_box_format=bounding_box_format,      # Match your format
        max_detections=300,            # Optional: max boxes per image
        from_logits=False              # Whether to apply sigmoid on confidence
    )
    
    # Tạo model với pre-trained backbone và custom head ---------------------------
    model = keras_cv.models.YOLOV8Detector(
        num_classes=num_classes,
        bounding_box_format=bounding_box_format,
        backbone=backbone,
        fpn_depth=fpn_depth,
        prediction_decoder=prediction_decoder,
        
    )

    # Đóng băng backbone (tùy chọn) -----------------------------------------------
    # Chỉ images các layer detection head
    backbone.trainable = trainable  # Đóng băng toàn bộ backbone

    # Hoặc đóng băng một phần:
    # for layer in backbone.layers[:-5]:  # Giữ lại 5 layer cuối để fine-tune
    #     layer.trainable = False

    # Cấu hình optimizer ----------------------------------------------------------
    optimizer = AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        global_clipnorm=10.0  # Thêm clipnorm để ổn định training
    )

    # Compile model ---------------------------------------------------------------
    model.compile(
        optimizer=optimizer,
        classification_loss=tf.keras.losses.BinaryCrossentropy(
            from_logits=True
        ),
        box_loss=keras_cv.losses.CIoULoss(
            bounding_box_format=bounding_box_format
        ),
        jit_compile=False
    )

    return model

In [6]:
with tf.device('/device:GPU:0'):
  model = create_model(num_classes = 1)

100%|██████████| 645/645 [00:00<00:00, 40.8kB/s]


100%|██████████| 5.11M/5.11M [00:02<00:00, 2.63MB/s]


Loaded pre-trained YOLOv8 backbone


In [7]:
model.summary()

Model: "yolov8_detector"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 640, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ [(None, 80, 80,   │  1,277,680 │ input_layer_1[0]… │
│ (Functional)        │ 64), (None, 40,   │            │                   │
│                     │ 40, 128), (None,  │            │                   │
│                     │ 20, 20, 256)]     │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat (Repeat)     │ (None, 40, 20,    │          0 │ functional[0][2]  │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_1 (Repeat)   │ (None, 40, 40,    │          0 │ repeat[0][0]      │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 40, 40,    │          0 │ repeat_1[0][0],   │
│ (Concatenate)       │ 384)              │            │ functional[0][1]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_co… │ (None, 40, 40,    │     49,152 │ concatenate_5[0]… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_bn  │ (None, 40, 40,    │        512 │ pa_fpn_p4p5_pre_… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre     │ (None, 40, 40,    │          0 │ pa_fpn_p4p5_pre_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ split_4 (Split)     │ [(None, 40, 40,   │          0 │ pa_fpn_p4p5_pre[… │
│                     │ 64), (None, 40,   │            │                   │
│                     │ 40, 64)]          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 42, 42,    │          0 │ split_4[0][1]     │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 40, 40,    │     36,864 │ pa_fpn_p4p5_pre_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 40, 40,    │        256 │ pa_fpn_p4p5_pre_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_1 │ (None, 40, 40,    │          0 │ pa_fpn_p4p5_pre_… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 42, 42,    │          0 │ pa_fpn_p4p5_pre_… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 40, 40,    │     36,864 │ pa_fpn_p4p5_pre_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pa_fpn_p4p5_pre_0_… │ (None, 40, 40,    │        256 │ pa_fpn_p4p5_pre_

 Total params: 3,432,563 (13.09 MB)

 Trainable params: 2,148,739 (8.20 MB)

 Non-trainable params: 1,283,824 (4.90 MB)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=96)

## Training

In [ ]:
# get best model
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'YOLOV8_model.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [ ]:
hist = model.fit(train_dataset, validation_data = valid_dataset,  epochs = EPOCHS, callbacks=[checkpoint, early_stop])

In [ ]:
# save model
model.save('YOLOV8_model.h5')

## Results and Evaluation

In [ ]:
fig, axs = plt.subplots(1,3, figsize = (18,5), dpi = 130)

axs[0].grid(linestyle="dashdot")
axs[0].set_title("Loss")
axs[0].plot(hist.history['loss'][1:])
axs[0].plot(hist.history['val_loss'][1:])
axs[0].legend(["images", "validataion"])

axs[1].grid(linestyle="dashdot")
axs[1].set_title("Box Loss")
axs[1].plot(hist.history['box_loss'])
axs[1].plot(hist.history['val_box_loss'])
axs[1].legend(["images",  "validataion"])

axs[2].grid(linestyle="dashdot")
axs[2].set_title("Class Loss")
axs[2].plot(hist.history['class_loss'][1:])
axs[2].plot(hist.history['val_class_loss'][1:])
axs[2].legend(["images",  "validataion"])